In [ ]:
%load_ext autoreload
%matplotlib widget

In [ ]:
%autoreload
from sisyphy.data_processing.rhd_reading import load_file
from pathlib import Path
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
data_path = Path("/Users/vigji/Downloads/barcode_testing_s3")
intan_path = data_path / "intan"

data_intan = []
time_intan = [[0]]
for f in sorted(intan_path.glob("*.rhd")):
    print(f)
    raw = load_file(f)
    data_intan.append(raw["board_dig_in_data"])
    time_intan.append(raw["t_dig"])  # + times_intan[-1][-1])
data_intan = np.concatenate(data_intan, 1)
time_intan = np.concatenate(time_intan[1:])

In [ ]:
plt.figure()
plt.plot(data_intan[1, :])

In [ ]:
barcode_array = data_intan[1, :].astype(int) * 5

In [ ]:
plt.figure()
plt.plot(barcode_array)

In [ ]:
DURATION_THR = 15  # duration in ms below which one pulse is considered a wrapper
BIT_WIDTH_MS = 30  # duration of each bit in ms
WRAPPER_WIDTH_MS = 10  # duration of the wrapper in ms
SIG_THR = 2.5  # threshold on signal for ON
DIFF_THR = 0.5  # threshold on diff for transition
FS_KHZ = 20.0  # sampling frequency in kHz
N_BITS = 32

wrap_width_pts = int(WRAPPER_WIDTH_MS * FS_KHZ)
bit_width_pts = int(BIT_WIDTH_MS * FS_KHZ)
duration_thr_pts = int(DURATION_THR * FS_KHZ)

bit_base = 2 ** np.arange(N_BITS, dtype=np.int64)

pulses_diff = np.diff(barcode_array)
event_indexes = np.argwhere(np.abs(pulses_diff) > DIFF_THR)[:, 0]

# Get on and off events:
on_events = pulses_diff[event_indexes] > DIFF_THR
off_events = ~on_events

wrapper_select = (
    on_events[:-1] & off_events[1:] & (np.diff(event_indexes) < duration_thr_pts)
)
wrapper_indexes = event_indexes[:-1][wrapper_select]

In [ ]:
plt.figure()
plt.plot(pulses_diff)

In [ ]:
bit_base = 2 ** np.arange(N_BITS, dtype=np.int64)

barcodes = []
for wrapper_idx in wrapper_indexes[:-1:2]:
    reading_start = wrapper_idx + wrap_width_pts * 2 + bit_width_pts // 2

    # Read voltage level in the middle of the bins (#TODO maybe an average would be better):
    bit_center_indexes = reading_start + np.arange(N_BITS) * bit_width_pts
    bool_bits = barcode_array[bit_center_indexes] > SIG_THR
    barcodes.append(sum(bit_base * bool_bits))

barcodes = np.array(barcodes)

In [ ]:
barcodes_times = time_intan[wrapper_indexes[:-1:2]]

### load csv

In [ ]:
import pandas as pd

In [ ]:
tstamps = pd.read_csv(next(data_path.glob("*timestamps.txt")))
tmouse = pd.read_csv(next(data_path.glob("*mouse.txt")))
# tstamps["t_ns_code"] = (tstamps["t_ns_code"] - tstamps["t_ns_code"][0]) / 1e9

# t_0_sp = tmouse["t_ns"][0]
# tstamps["t_ns"] = (tstamps["t_ns"] - t_0_sp) / 1e9
# tmouse["t_ns"] = (tmouse["t_ns"] - t_0_sp) / 1e9

In [ ]:
intan_barcodes = pd.DataFrame(dict(t_s=barcodes_times), index=barcodes)
tstamps = tstamps.set_index("code")

In [ ]:
pairs = []

for bcode in tstamps.index:
    pairs.append((tstamps.loc[bcode, "t_ns_code"], intan_barcodes.loc[bcode, "t_s"]))

pairs = np.array(pairs)

In [ ]:
pairs = pairs - pairs[0, :]

In [ ]:
f, axs = plt.subplots(2, 1, sharex=True, figsize=(4, 3))
x = np.arange(-6, 6, 0.2)
for arr, ax in zip([intan_barcodes["t_s"], tstamps["t_ns_code"] / 1e9], axs):
    ax.hist((np.diff(arr) - np.mean(np.diff(arr))) * 1000, x, histtype="step")
    sd = np.std((np.diff(arr) - np.mean(np.diff(arr))) * 1000)
    # ax.text(0, 50, sd)
    ax.set(ylabel="Count")
ax.set(xlabel="Jitter (ms)")
plt.show()
plt.tight_layout()

In [ ]:
main_numpy_barcode = intan_barcodes.index.values  # main_numpy_data[barcodes_row, :]
secondary_numpy_barcode = tstamps.index.values  # secondary_numpy_data[barcodes_row, :]

main_numpy_timestamp = intan_barcodes[
    "t_s"
].values  # main_numpy_data[barcode_timestamps_row, :]
secondary_numpy_timestamp = tstamps[
    "t_ns_code"
].values  # secondary_numpy_data[barcode_timestamps_row, :]

# Pull the index values from barcodes shared by both groups of data
shared_barcodes, main_index, second_index = np.intersect1d(
    main_numpy_barcode, secondary_numpy_barcode, return_indices=True
)

# Use main_index and second_index arrays to extract related timestamps
main_shared_barcode_times = main_numpy_timestamp[main_index]
secondary_shared_barcode_times = secondary_numpy_timestamp[second_index]

# Determine slope (m) between main/secondary timestamps
m = (main_shared_barcode_times[-1] - main_shared_barcode_times[0]) / (
    secondary_shared_barcode_times[-1] - secondary_shared_barcode_times[0]
)
# Determine offset (b) between main and secondary barcode timestamps
b = main_shared_barcode_times[0] - secondary_shared_barcode_times[0] * m

In [ ]:
mouse_signal = tmouse.copy()
mouse_signal = mouse_signal.set_index("t_ns").drop("Unnamed: 0", axis=1)
mouse_signal.index = pd.to_datetime(mouse_signal.index, unit="ns")
mouse_signal.resample("1ms").mean().interpolate()
mouse_signal.index = mouse_signal.index.astype(np.int64)

In [ ]:
fs_intan = 8000


def _convolve(sig, fs, kernel_size_s=0.05):
    kernel_size_pts = int(kernel_size_s * fs)
    print(kernel_size_pts)
    kernel = np.ones(kernel_size_pts) / kernel_size_pts
    return np.convolve(sig, kernel, mode="same")  # [:len(sig) + 1]


sig_intan = _convolve(np.diff(data_intan[5:7, :], axis=1).sum(0), 8000)

converted_tscale = mouse_signal.index * m + b
sig_mouse = np.abs(mouse_signal[["pitch", "roll", "yaw"]].values.mean(1))
sig_mouse = _convolve(sig_mouse, 1000)

In [ ]:
np.nanpercentile(sig_intan, 99.9)

In [ ]:
f, ax = plt.subplots(figsize=(8, 3))
ax.plot(
    converted_tscale, sig_mouse / np.percentile(sig_mouse, 99.9), label="Mouse speed"
)
slice_t = slice(64562000, 66662000)
ax.plot(
    time_intan[slice_t],
    sig_intan[slice_t] / np.nanpercentile(sig_intan, 99.9),
    label="Intan rotarod speed",
)
ax.set(xlabel="Recording time (s)", xlim=(3271, 3274))
ax.legend(frameon=False)
plt.tight_layout()

In [ ]:
len(time_intan) / 22000

In [ ]:
time_intan[-1]

In [ ]:
3281 * 22000

In [ ]:
secondary_data_original[:, convert_timestamp_column] = (
    secondary_data_original * secondary_sample_rate * m + b
)

# Clean up conversion of values to nearest whole number
# print("Total number of index values: ", len(secondary_data_converted[:,convert_timestamp_column]))
value = secondary_data_converted[index, convert_timestamp_column]
secondary_data_converted[convert_timestamp_column] = value.astype("int")

In [ ]:
"""
  Optogenetics and Neural Engineering Core ONE Core
  University of Colorado, School of Medicine
  31.Oct.2021
  See bit.ly/onecore for more information, including a more detailed write up.
  alignment_barcodes.py
################################################################################
  This code takes two Numpy files ("main" and "secondary" data) that contain the 
  timestamp and barcode values collected using "extraction_barcodes.py", and 
  finds the linear conversion variables needed to align the timestamps. Then it
  takes the original (or .npy converted) secondary data file, applies the linear
  conversion to its timestamp column, and outputs this data as a Numpy (.npy) or
  CSV (.csv) file into a chosen directory.
################################################################################
  USER INPUTS EXPLAINED:

  Input Variables:
  = main_sample_rate = (int) The sampling rate (in Hz) of the "main" DAQ, to
                       which the secondary data will be aligned.
  = secondary_sample_rate = (int) The sampling rate (in Hz) of the "secondary"
                            DAQ, which will be aligned to the "main" DAQ.
  = convert_timestamp_column = (int) The timestamp column in the original 
                               "secondary" data file; this will be converted to
                               match the timestamps in the original "main" data.

  Output Variables: 
  = alignment_name = (str) The name of the file(s) in which the output will be 
                     saved in the chosen directory.
  = save_npy = (bool) Set to "True" to save the aligned data as a .npy file.
  = save_csv = (bool) Set to "True" to save the aligned data as a .csv file.
  
################################################################################
  References

"""
#######################
### Imports Section ###
#######################

import sys
import numpy as np
from datetime import datetime
from pathlib import Path
from tkinter.filedialog import askdirectory, askopenfilename

################################################################################
############################ USER INPUT SECTION ################################
################################################################################

# Input variables
main_sample_rate = 30000  # Expected sample rate of main data, in Hz
secondary_sample_rate = 2000  # Expected sample rate of secondary data, in Hz
convert_timestamp_column = 0  # Column that timestamps are located in secondary data

# Output variables
alignment_name = "LabJackAlignedToNeuroPixelTest"  # Name of output file.
save_npy = False  # Save output file as a .npy file
save_csv = False  # Save output file as a .csv file

################################################################################
############################ END OF USER INPUT #################################
################################################################################

##########################################
### Select Files for Barcode Alignment ###
##########################################

# Have user select files to be used.
# Main data's barcodes input file:
try:
    main_dir_and_name = Path(askopenfilename(title="Select Main Data Barcodes File"))
except:
    print("No Main Data Barcodes File Chosen")
    sys.exit()
# Secondary data's barcodes input file
try:
    secondary_dir_and_name = Path(
        askopenfilename(title="Select Secondary Data Barcodes File")
    )
except:
    print("No Secondary Data Barcodes File Chosen")
    sys.exit()
# Secondary data file to be aligned with main data.
try:
    secondary_raw_data = Path(
        askopenfilename(title="Select Secondary Data File to Align")
    )
except:
    print("No Secondary Data File to Align Chosen")
    sys.exit()

# Try to load the selected files; if they fail, inform the user.
try:
    main_numpy_data = np.load(main_dir_and_name)
except:
    main_numpy_data = ""
    print("Main .npy file not located/failed to load; please check the filepath")

try:
    secondary_numpy_data = np.load(secondary_dir_and_name)
except:
    secondary_numpy_data = ""
    print("Secondary .npy file not located/failed to load; please check the filepath")

try:
    secondary_data_original = np.load(secondary_raw_data)
except:
    secondary_data_original = ""
    print(
        "Data file to be aligned not located/failed to load; please check your filepath"
    )

# Have user select folder into which aligned data will be saved; if no format
# selected, inform the user.
if save_npy or save_csv or save_dat:
    try:
        alignment_dir = Path(askdirectory(title="Select Folder to Save Aligned Data"))
    except:
        print("No Output Directory Selected")
        sys.exit()
else:
    print("Aligned data will not be saved to file in any format.")


##########################################################################
### Extract Barcodes and Index Values, then Calculate Linear Variables ###
##########################################################################

# Pull the barcode row from the data. 1st column is timestamps, second is barcodes
barcode_timestamps_row = (
    0  # Same for both main and secondary, because we used our own code
)
barcodes_row = 1  # Same for both main and secondary

main_numpy_barcode = main_numpy_data[barcodes_row, :]
secondary_numpy_barcode = secondary_numpy_data[barcodes_row, :]

main_numpy_timestamp = main_numpy_data[barcode_timestamps_row, :]
secondary_numpy_timestamp = secondary_numpy_data[barcode_timestamps_row, :]

# Pull the index values from barcodes shared by both groups of data
shared_barcodes, main_index, second_index = np.intersect1d(
    main_numpy_barcode, secondary_numpy_barcode, return_indices=True
)
# Note: To intersect more than two arrays, use functools.reduce

# Use main_index and second_index arrays to extract related timestamps
main_shared_barcode_times = main_numpy_timestamp[main_index]
secondary_shared_barcode_times = secondary_numpy_timestamp[second_index]

# Determine slope (m) between main/secondary timestamps
m = (main_shared_barcode_times[-1] - main_shared_barcode_times[0]) / (
    secondary_shared_barcode_times[-1] - secondary_shared_barcode_times[0]
)
# Determine offset (b) between main and secondary barcode timestamps
b = main_shared_barcode_times[0] - secondary_shared_barcode_times[0] * m

print("Linear conversion from secondary timestamps to main:\ny = ", m, "x + ", b)

##################################################################
### Apply Linear Conversion to Secondary Data (in .npy Format) ###
##################################################################

secondary_data_original[:, convert_timestamp_column] = (
    secondary_data_original[:, convert_timestamp_column] * secondary_sample_rate * m + b
)
secondary_data_converted = secondary_data_original  # To show conversion complete.

# Clean up conversion of values to nearest whole number
# print("Total number of index values: ", len(secondary_data_converted[:,convert_timestamp_column]))
for index in range(0, len(secondary_data_converted[:, convert_timestamp_column])):
    value = secondary_data_converted[index, convert_timestamp_column]
    rounded_val = value.astype("int")
    secondary_data_converted[index, convert_timestamp_column] = rounded_val

################################################################
### Print out final output and save to chosen file format(s) ###
################################################################

# Test to see output here:
print("Final output for LJ Data:\n", secondary_data_converted)

time_now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

if save_npy:
    output_file = alignment_dir / (alignment_name + time_now)
    np.save(output_file, secondary_data_converted)

if save_csv:
    output_file = alignment_dir / (alignment_name + time_now + ".csv")
    np.savetxt(output_file, secondary_data_converted, delimiter=",", fmt="%s")